In [ ]:
!pip install pyopencl

### Kernel

In [ ]:
%%writefile program.cl

__kernel void integral_sum(__global float* dx, __global float* sums) {
	int id = get_global_id(0);
	int local_id = get_local_id(0);
	int group_id = get_group_id(0);
	int group_size = get_local_size(0);
	int i;

	float sum = 0.0f;

	float x = dx[0] * id;
	sums[id] = 4 / (1 + pow(x, 2.0f)) * dx[0];
}

### Runtime

In [ ]:
import numpy as np
import pyopencl as cl

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

STEP = 0.0001
NUM_SUMS = int(1/STEP)

dx = np.array([STEP], dtype=np.float32)
sums = np.zeros((NUM_SUMS,), dtype=np.float32)

mf = cl.mem_flags
dx_buf = cl.Buffer(ctx, mf.READ_ONLY, size=dx.nbytes)
sums_buf = cl.Buffer(ctx, mf.WRITE_ONLY, size=sums.nbytes)

cl.enqueue_copy(queue, dx_buf, dx)
cl.enqueue_copy(queue, sums_buf, sums)
queue.finish()

program_file = open("program.cl", "r")
program_src = program_file.read()

program = cl.Program(ctx, program_src)
program.build()
integral_sum = program.integral_sum

integral_sum(queue, (NUM_SUMS,), None, dx_buf, sums_buf)

cl.enqueue_copy(queue, sums, sums_buf)
print("π =", np.sum(sums))